In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [2]:
!pip install torch torchvision torchaudio transformers datasets matplotlib scikit-learn


In [3]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


PyTorch version: 2.4.1+cu121
CUDA available: False


In [6]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
print("Scikit-learn está instalado y listo para usar.")


Scikit-learn está instalado y listo para usar.


In [7]:
# Instalar librerías necesarias
!pip install transformers datasets matplotlib
import torch
import time
import os

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
from datasets import load_dataset
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, TrainerCallback


/anaconda/envs/azureml_py38/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/anaconda/envs/azureml_py38/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
2025-01-21 02:29:49.730208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already be

In [8]:
# 0. Deshabilitar WandB
os.environ["WANDB_DISABLED"] = "true"

In [9]:
# 1. Cargar el dataset IMDB
dataset = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:

# 2. Configurar el tokenizador
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )


/anaconda/envs/azureml_py38/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [11]:

# 3. Tokenización optimizada
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    batch_size=1000,
    num_proc=4
)

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/25000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [12]:

# Preparar el dataset para PyTorch
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["test"]


In [13]:

# 4. Cargar el modelo BERT preentrenado
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:

# 5. Argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",               # Carpeta de salida
    evaluation_strategy="epoch",         # Evaluar en cada época
    logging_dir="./logs",                # Carpeta de logs
    logging_steps=50,                    # Frecuencia de registro de métricas
    learning_rate=2e-5,                  # Tasa de aprendizaje
    per_device_train_batch_size=8,       # Tamaño de lote de entrenamiento
    per_device_eval_batch_size=8,        # Tamaño de lote de evaluación
    num_train_epochs=1,                  # Número de épocas
    weight_decay=0.01,                   # Decaimiento del peso
    save_total_limit=2,                  # Guardar máximo 2 checkpoints
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [25]:
# 6. Personalizar el callback para registrar métricas
class MetricsLogger(TrainerCallback):
    def __init__(self):
        self.logs = {"train_loss": [], "eval_loss": [], "epoch": []}

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return
        # Registrar las métricas
        if "loss" in logs:
            self.logs["train_loss"].append(logs["loss"])
        if "eval_loss" in logs:
            self.logs["eval_loss"].append(logs["eval_loss"])
        if "epoch" in logs:
            self.logs["epoch"].append(logs["epoch"])

metrics_logger = MetricsLogger()

In [26]:
# Función para calcular métricas
def compute_metrics(pred):
    predictions = pred.predictions.argmax(-1)  # Obtener las clases predichas
    labels = pred.label_ids                   # Etiquetas reales
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [27]:
# Configurar el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Añadir cálculo de métricas
)

In [28]:
# 8. Entrenar el modelo
trainer.train()


2025/01/21 02:33:49 INFO mlflow.tracking._tracking_service.client: 🏃 View run ./results at: https://eastus.api.azureml.ms/mlflow/v2.0/subscriptions/ff6dd32f-ba95-466d-8b04-529c86f648b6/resourceGroups/kenmachinelearning/providers/Microsoft.MachineLearningServices/workspaces/kenbert/#/experiments/741cb9aa-5217-446a-ae51-82fa4e05c45e/runs/f276f2c8-dda8-420f-8e55-884231c957b8.
2025/01/21 02:33:49 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://eastus.api.azureml.ms/mlflow/v2.0/subscriptions/ff6dd32f-ba95-466d-8b04-529c86f648b6/resourceGroups/kenmachinelearning/providers/Microsoft.MachineLearningServices/workspaces/kenbert/#/experiments/741cb9aa-5217-446a-ae51-82fa4e05c45e.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.289700,0.307914,0.886640,0.880312,0.894960,0.887575


2025/01/21 04:11:50 INFO mlflow.tracking._tracking_service.client: 🏃 View run ./results at: https://eastus.api.azureml.ms/mlflow/v2.0/subscriptions/ff6dd32f-ba95-466d-8b04-529c86f648b6/resourceGroups/kenmachinelearning/providers/Microsoft.MachineLearningServices/workspaces/kenbert/#/experiments/741cb9aa-5217-446a-ae51-82fa4e05c45e/runs/3fcc5af7-c856-4c43-a807-7b326b9dc91b.
2025/01/21 04:11:50 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://eastus.api.azureml.ms/mlflow/v2.0/subscriptions/ff6dd32f-ba95-466d-8b04-529c86f648b6/resourceGroups/kenmachinelearning/providers/Microsoft.MachineLearningServices/workspaces/kenbert/#/experiments/741cb9aa-5217-446a-ae51-82fa4e05c45e.


TrainOutput(global_step=3125, training_loss=0.34871707244873046, metrics={'train_runtime': 5880.1166, 'train_samples_per_second': 4.252, 'train_steps_per_second': 0.531, 'total_flos': 1644444096000000.0, 'train_loss': 0.34871707244873046, 'epoch': 1.0})

In [29]:
# Guardar el modelo entrenado
output_dir = "./imdb_bert_model"
trainer.save_model(output_dir)

# Guardar el tokenizador asociado al modelo
tokenizer.save_pretrained(output_dir)

print(f"Modelo y tokenizador guardados en: {output_dir}")


Modelo y tokenizador guardados en: ./imdb_bert_model


In [30]:

# Evaluar el modelo en el conjunto de validación
eval_results = trainer.evaluate()

# Mostrar las métricas de evaluación
print("Resultados de la evaluación:")
for key, value in eval_results.items():
    print(f"{key}: {value}")

# Tamaño del modelo en MB
model_size = sum(p.numel() for p in model.parameters()) * 4 / (1024 ** 2)  # 4 bytes por float32

# Número de parámetros entrenables
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)


Resultados de la evaluación:
eval_loss: 0.3079136610031128
eval_accuracy: 0.88664
eval_precision: 0.8803116147308782
eval_recall: 0.89496
eval_f1: 0.8875753728974929
eval_runtime: 1167.8566
eval_samples_per_second: 21.407
eval_steps_per_second: 2.676
epoch: 1.0


In [33]:
import time
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Ruta donde guardaste el modelo
output_dir = "./imdb_bert_model"

# Cargar el modelo y el tokenizador
model = AutoModelForSequenceClassification.from_pretrained(output_dir)  # Cargar modelo
tokenizer = AutoTokenizer.from_pretrained(output_dir)  # Cargar el tokenizador

# Configurar el dispositivo (CPU)
device = torch.device("cpu")
model.to(device)  # Asegurarse de que el modelo esté en CPU

# Texto de prueba
text = "This movie was absolutely horrible! I loved every moment of it."

# Tokenizar el texto de prueba
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

# Asegurar que los datos también estén en CPU
inputs = {key: value.to(device) for key, value in inputs.items()}

# Medir el tiempo de inferencia
model.eval()  # Cambiar el modelo a modo de evaluación
start_time = time.time()  # Iniciar medición de tiempo
with torch.no_grad():
    outputs = model(**inputs)
end_time = time.time()  # Finalizar medición de tiempo

# Calcular el tiempo de inferencia
inference_time = (end_time - start_time) * 1000  # Tiempo en milisegundos

# Obtener la clase predicha
predicted_class = torch.argmax(outputs.logits, dim=-1).item()

# Mostrar el resultado
print(f"Predicción: {'Positivo' if predicted_class == 1 else 'Negativo'}")
print(f"Tiempo de inferencia en CPU: {inference_time:.2f} ms")


Predicción: Negativo
Tiempo de inferencia en CPU: 35.17 ms
